In [1]:
!wget https://storage.yandexcloud.net/datasouls-ods/materials/acfacf11/train_matching.csv

--2022-05-21 16:46:23--  https://storage.yandexcloud.net/datasouls-ods/materials/acfacf11/train_matching.csv
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1070145 (1.0M) [text/csv]
Saving to: ‘train_matching.csv’

train_matching.csv  100%[===================>]   1.02M  1.05MB/s    in 1.0s    

2022-05-21 16:46:25 (1.05 MB/s) - ‘train_matching.csv’ saved [1070145/1070145]



In [2]:
!pip install catboost

     |████████████████████████████████| 76.6 MB 87 kB/s 


In [3]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, Pool
import pickle
from tqdm import tqdm

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
train_matching = pd.read_csv('./train_matching.csv')

In [6]:
def dtypeList(data_frame):
    dtype_list = list()
    for x in data_frame.dtypes.tolist():
        if x=='int64':
            dtype_list.append('int16')
        elif(x=='float64'):
            dtype_list.append('float32')
        else:
            dtype_list.append('object')
            
    return dict(zip(data_frame.columns.tolist(),dtype_list))

In [7]:
train = pd.read_csv('/content/drive/MyDrive/data/train.csv', nrows=1).fillna(0)
test = pd.read_csv('/content/drive/MyDrive/data/test.csv', nrows=1).fillna(0)

train = pd.read_csv('/content/drive/MyDrive/data/train.csv', dtype=dtypeList(train)).fillna(0)
test = pd.read_csv('/content/drive/MyDrive/data/test.csv', dtype=dtypeList(test)).fillna(0)

In [8]:
train_data = Pool(train[train.columns[3:]],label=train['target'], has_header=True)
eval_data = Pool(test[test.columns[3:]], label=test['target'], has_header=True)

In [9]:
full_list_of_features = list(train.columns[3:])

In [14]:
model = CatBoostClassifier(
        iterations=6000, depth=6, learning_rate=0.01, loss_function='CrossEntropy')

In [15]:
m = model.fit(train_data, eval_set=eval_data, verbose=300, early_stopping_rounds=10)

0:	learn: 0.6828713	test: 0.6829686	best: 0.6829686 (0)	total: 549ms	remaining: 54m 55s
300:	learn: 0.2721023	test: 0.2737823	best: 0.2737823 (300)	total: 3m	remaining: 56m 54s
600:	learn: 0.2665357	test: 0.2690059	best: 0.2690052 (599)	total: 6m 16s	remaining: 56m 19s
900:	learn: 0.2636165	test: 0.2669771	best: 0.2669759 (897)	total: 9m 19s	remaining: 52m 47s
1200:	learn: 0.2606767	test: 0.2651237	best: 0.2651237 (1200)	total: 12m 29s	remaining: 49m 54s
1500:	learn: 0.2562006	test: 0.2619916	best: 0.2619916 (1500)	total: 16m 9s	remaining: 48m 27s
1800:	learn: 0.2527564	test: 0.2596882	best: 0.2596882 (1800)	total: 19m 36s	remaining: 45m 43s
2100:	learn: 0.2499928	test: 0.2580862	best: 0.2580862 (2100)	total: 22m 54s	remaining: 42m 31s
2400:	learn: 0.2476647	test: 0.2569350	best: 0.2569350 (2400)	total: 26m 8s	remaining: 39m 11s
2700:	learn: 0.2456184	test: 0.2559932	best: 0.2559928 (2699)	total: 29m 15s	remaining: 35m 44s
3000:	learn: 0.2437803	test: 0.2551965	best: 0.2551965 (3000)	t

In [16]:
model.save_model('model.cbm')

In [17]:
with open("full_list_of_features", "wb") as fp:   #Pickling
    pickle.dump(full_list_of_features, fp)